In [1]:
# File Name: SageCheckIWnkViaILPAgainstSignedPermutationsDate29082022Ver01.txt
# Purpose: Check (Integer) Weighing matrices Via Integer Linear Programming against (direct) Signed permutations
# Remark: After session with Radi. We are left with a question:
#         Why No solution for R1,R2:  [0, 0, 0, 0, 0, 0, 5] [0, 0, 2, 2, 2, 2, 3]
# Improvements: (1) Problem from remark fixed.
#               (2) Added "sors" routine from Assaf, so to generalize the whole activity.
# Remark: Radi on 8/5/22 noticed that solutions begin with negative values although BegWithPos should prevent it.
#         We fix it.
# Remark: On 12/5/22 we added two routines
#         OurMain2: Given parameters of weighing matrix, weight and 2 divisors of order, it gives all pairs of
#                   orthogonal rows of all types available of nsoks and filter them by the code invariany method.
#         AddLine:  Given Nsoks and Sols gives all additions to Sols by one row taken from Nsoks
#                   with permutations and signings and codeinvariant.
#                   This routine is the basis for recursion to construct all Integer Weighing Matrices.
#                   We have not run and debugged this routine yet. Next time...
# Remark: On 15/5/22 we wroteqimproved/touched some routines:
#         OurMain2: This is the new main. It gets the [WMweight, WMOrderDiv] of a weighing matrix and should output all inequivalent Integer weighing matrices of order=WMOrderDiv & weight=sqrt(WMweight. 
#         FilterOrthogonalSignedPermutationsAgainstMatrix (haven't debugged yet)
#         CodePrevRows (debugged carefully)
# Remark: On 26052022 we debuged OurMain2 routine.
# Remark On 30-6-22 Organizing the hp check. 
#                   automatic base calculation for the code invariant
#                   indexing the permutation and signs and saving the index in the configuration.
# Remark on 3-7-22 Adding code to check for Hadamrd equivalnce for two rows for permutation AND row negation
#                  Changes in "FilterOrthogonalSignedPermutationsByCodeInvariant" adding two code invariants for each line.
# Remark on 14-7-22 Adding the index for each line of (NsoksIndex,PermutationIndex,SignsIndex) 
#                    Bug for line [3, 2, 2, 2, 2, 0, 0], [[1, 16, 5], (2, -3, 2, -2, 0, 2, 0)]], 
#                                 The SignsIndex looks incorrect - TBD in the next meeting
# Remark on 21-7-22 Generelizing for AboveMatrix.
#                    -   OrthogonalSignedPermutationsWithFirstNonZeroPositiveAgainstMatrix
#                    -   normAboveAgainstMatrix(AboveMatrix,R2):
#                    -   dpAgainstMatrix
#                       need to correct FilterOrthogonalSignedPermutationsByCodeInvariantAgainstMatrix
# Remark on 24-7-22  began adapting FilterOrthogonalSignedPermutationsByCodeInvariantAgainstMatrix
#                    adding routine genCode for generating a code for the column i.
#                   for next time - continue to adapt  FilterOrthogonalSignedPermutationsByCodeInvariantAgainstMatrix
#                                    and remove CodePrevRows
# Remark on 31-7-22 remove CodePrevTows and AddLine
#                   Checked FilterOrthogonalSignedPermutationsByCodeInvariantAgainstMatrix for addition of one row.
#                   Coded OurMain3 to work with FilterOrthogonalSignedPermutationsByCodeInvariantAgainstMatrix
#                   Checked OurMain3 for two lines.
# Remark on 31-7-22 Adapt OurMain3 for 7 lines
#                   While running OurMain3 it works fine for 2 and 3 lines but crashes while
#                             calculating for 4 lines in memory shortage
# Remark on 18-7-22 Cleanup
# Remark on 28-8-22 
#                  integrate   FilterOrthogonalSignedPermutationsByCodeInvariant
#                           to OrthogonalSignedPermutationsWithFirstNonZeroPositiveAgainstMatrix
#                  integrate hp into OrthogonalSignedPermutationsWithFirstNonZeroPositiveAgainstMatrix
#                  integrate gencode also
#                  add goodPerm test to eliminate (-1) assignment to 0 (faster)


def HadamardSpace(d):
    SpaceSize = 2^d
    L=[ZZ(n).digits(2) for n in range(SpaceSize)]
    L1 = [x+[0]*(d-len(x)) for x in L]
    return([vector([2*y-1 for y in x]) for x in L1])

def IsBegWithPos(L):
    for x in L:
        if x>0:
            return(True)
        if x<0:
            return(False)
    return(False)

def goodPerm(L,s):
    for i in range(len(L)):
        if (L[i]==0) and (s[i]==-1) : return false
    return true

def normAboveAgainstMatrix(AboveMatrix,R2):
    for i in range(len(R2)):
        if AboveMatrix.column(i).norm()==0:
            R2[i]=abs(R2[i])
    return R2


def dpAgainstMatrix(AboveMatrix,L2):
    return((AboveMatrix*vector(L2)).norm())


def OrthogonalSignedPermutationsWithFirstNonZeroPositiveAgainstMatrix(AboveMat,L,Lindex,MaxV):
    P = Permutations(L).list()
    d = len(L)
    S = HadamardSpace(d)
    SP = []
    s = []
    sd = [] 
    codeBase = 1+2*MaxV
    nR=AboveMat.nrows()
    cB=(vector([codeBase^(nR-j) for j in range(nR)]))*AboveMat
    for pi in range(Lindex[1],len(P)):
        p = P[pi]
        s0=Lindex[2] if pi==Lindex[1] else 0
        for si in range(s0,len(S)):
            sn=S[si]
            if (goodPerm(p,sn)):
                r =[p[i]*sn[i] for i in range(d)]
                if IsBegWithPos(r):
                    if dpAgainstMatrix(AboveMat,r)==0:
                        r=normAboveAgainstMatrix(AboveMat,r)
                        if not(r in SP):
                            SP.append(r)
                            R2i=vector(r)
                            M1 = cB+R2i
                            M1s = sorted(M1)
                            if not(M1s in s) :
                                M1m = cB-R2i
                                M1sm = sorted(M1m)
                                if not(M1sm in s):
                                    s.append(M1s)
                                    sd.append([[Lindex[0],pi,si],R2i])                    
    return(sd)



def sors(n,r,maxsq):  ### Find all ways to express n=\sum_i s_i^2 as a sum of r squares, with maxsq>=s_1>=s_2>=...>=s_r.
    #n0=RDF(n)
    n0=n
    if r==1:
        s=sqrt(n0)
        fs=floor(s)
        if s==fs and s<=maxsq:
            return [[s]]
        else:
            return false
    else:
        sors2=[]
        for x in range(floor(sqrt(n0/r)),min(floor(sqrt(n0)),maxsq)+1):
            m=n-x^2
            #print x
            sors1=sors(m,r-1,x)
            if sors1:
                sors2+=[[x]+sors1[i] for i in range(len(sors1))   ]
           # print "sors2=",sors2
        if len(sors2)>0:
            return sors2
        else:
            return false


def OurMain3(WMWeight, WMOrderDiv,lines):
    MaxSquare = WMWeight^(1/2)
    AllSoks = sors(WMWeight, WMOrderDiv, MaxSquare)
    print("Nsoks: ",AllSoks)
    Sols= [[matrix(1,WMOrderDiv,AllSoks[i]),[i,0,0]] for i in range(len(AllSoks))]
    print(Sols)
    for line in range(lines-1):
        nSols=[]
        countSol=0
        countScan=0
        for Sol in Sols:
            countScan+=1
            AboveMat=Sol[0]
            R1Index=Sol[1]
            for R2i in range(R1Index[0],len(AllSoks)):
                R2Index=R1Index if R2i==R1Index[0] else [R2i,0,0]
                R2 = vector(AllSoks[R2i])
                cpF = OrthogonalSignedPermutationsWithFirstNonZeroPositiveAgainstMatrix(AboveMat,R2,R2Index,MaxSquare)
                countSol+=len(cpF)
                if len(cpF)>0:
                    nSols.extend([AboveMat,x] for x in cpF)
            print("countScan,countSol  = ",countScan,countSol)
        Sols=[]
        for nSol in nSols:
            NL=matrix(WMOrderDiv,1,list(nSol[1][1]))
            AM=((nSol[0].transpose()).augment(NL)).transpose()
            Sols.append([AM,nSol[1][0]])
        print("# of sols in line",line, "  =",len(Sols),[Sols[i] for i in range(min(len(Sols),3))])
    return(Sols)

from gc import collect
collect()
#%time res=OurMain3(8,8,8)

7

In [89]:
save (res,'IW(8,8)')

In [90]:
res = load('IW(8,8)')

In [92]:
res

[[
[ 1  1  1  1  1  1  1  1]             
[ 1  1  1  1 -1 -1 -1 -1]             
[ 1  1 -1 -1  1  1 -1 -1]             
[ 1 -1  1 -1  1 -1  1 -1]             
[ 1 -1 -1  1 -1  1  1 -1]             
[ 1 -1 -1  1  1 -1 -1  1]             
[ 1 -1  1 -1 -1  1 -1  1]             
[ 1  1 -1 -1 -1 -1  1  1], [0, 0, 195]
],
 [
[ 1  1  1  1  1  1  1  1]               
[ 1  1  1  1 -1 -1 -1 -1]               
[ 1  1 -1 -1  1  1 -1 -1]               
[ 1 -1  1 -1  1 -1  1 -1]               
[ 2 -1 -1  0 -1  0  0  1]               
[ 0  1  1 -2 -1  0  0  1]               
[ 0  1 -1  0  1 -2  0  1]               
[ 0  1 -1  0 -1  0  2 -1], [1, 225, 107]
],
 [
[ 1  1  1  1  1  1  1  1]               
[ 1  1  1  1 -1 -1 -1 -1]               
[ 1  1 -1 -1  1  1 -1 -1]               
[ 1 -1  1 -1  1 -1  1 -1]               
[ 1 -2  0  1  0  1 -1  0]               
[ 1  0 -2  1  0 -1  1  0]               
[ 1  0  0 -1 -2  1  1  0]               
[ 1  0  0 -1  0 -1 -1  2], [1, 170, 151]
],
 [
[ 1  1  1  

In [126]:
L=8*[1,-1]

In [127]:
p=Permutations(L).list()
len(p)

12870

In [3]:
%time res=OurMain3(16,6,6)

Nsoks:  [[2, 2, 2, 2, 0, 0], [3, 2, 1, 1, 1, 0], [4, 0, 0, 0, 0, 0]]
[[[2 2 2 2 0 0], [0, 0, 0]], [[3 2 1 1 1 0], [1, 0, 0]], [[4 0 0 0 0 0], [2, 0, 0]]]
countScan,countSol  =  1 9
countScan,countSol  =  2 20
countScan,countSol  =  3 21
# of sols in line 0   = 21 [[[ 2  2  2  2  0  0]
[ 2  2 -2 -2  0  0], [0, 0, 51]], [[ 2  2  2  2  0  0]
[ 2 -2  0  0  2  2], [0, 5, 13]], [[ 2  2  2  2  0  0]
[ 3 -2 -1  0  1  1], [1, 2, 9]]]
countScan,countSol  =  1 4
countScan,countSol  =  2 10
countScan,countSol  =  3 12
countScan,countSol  =  4 14
countScan,countSol  =  5 16
countScan,countSol  =  6 18
countScan,countSol  =  7 20
countScan,countSol  =  8 22
countScan,countSol  =  9 23
countScan,countSol  =  10 27
countScan,countSol  =  11 31
countScan,countSol  =  12 34
countScan,countSol  =  13 37
countScan,countSol  =  14 40
countScan,countSol  =  15 42
countScan,countSol  =  16 45
countScan,countSol  =  17 48
countScan,countSol  =  18 48
countScan,countSol  =  19 48
countScan,countSol  =  20 48
c

In [4]:
res

[[
[ 2  2  2  2  0  0]            
[ 2  2 -2 -2  0  0]            
[ 2 -2  2 -2  0  0]            
[ 2 -2 -2  2  0  0]            
[ 0  0  0  0  4  0]            
[ 0  0  0  0  0  4], [2, 5, 63]
],
 [
[ 2  2  2  2  0  0]             
[ 2  2 -2 -2  0  0]             
[ 2 -2  0  0  2  2]             
[ 2 -2  0  0 -2 -2]             
[ 0  0  2 -2  2 -2]             
[ 0  0  2 -2 -2  2], [0, 14, 39]
],
 [
[ 2  2  2  2  0  0]              
[ 2 -2  0  0  2  2]              
[ 2  0 -3  1 -1 -1]              
[ 2  0  1 -3 -1 -1]              
[ 0  2 -1 -1  3 -1]              
[ 0  2 -1 -1 -1  3], [1, 107, 35]
],
 [
[ 2  2  2  2  0  0]              
[ 2 -2  0  0  2  2]              
[ 2  0 -1 -1 -3  1]              
[ 2  0 -1 -1  1 -3]              
[ 0  2 -3  1  1  1]              
[ 0  2  1 -3  1  1], [1, 105, 55]
],
 [
[ 2  2  2  2  0  0]            
[ 3 -1 -1 -1  2  0]            
[ 1 -3  1  1 -2  0]            
[ 1  1 -3  1 -2  0]            
[ 1  1  1 -3 -2  0]            
[ 0  0  0  0  0

In [98]:
p[0][0]

1

In [105]:
p=[i^2  for i in range(10)]

In [118]:
from operator import mul
r=list(map(mul,p,p))
print(r)

[0, 1, 16, 81, 256, 625, 1296, 2401, 4096, 6561]
